In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import re

In [ ]:
# read in csv file
all_cases = gpd.read_file('/Users/leahwallihan/Durham_school_planning/Res_Development')

In [ ]:
# types = A_TYPE codes we want to keep
types = ['PL_MINSP', 'PL_SSP_SM', 'PL_SSM_SM2', 'PL_CPAA', 'PL_MINPP', 'PL_MAJSP', 'PL_MAJSUP', 'PL_PPA', 'PL_MAJPP'] 
# filter that only keeps cases of specified types
filter_cases_type = all_cases[all_cases['A_TYPE'].isin(types)]
# status = A_STATUS codes we want to keep
status = all_cases['A_STATUS'].unique()
status = status[~np.isin(status, ['WITH', 'VOID','DEN','DISAP','EXP'])]
# filter that only keeps cases of specified status
filter_cases_status = filter_cases_type[filter_cases_type['A_STATUS'].isin(status)]

In [ ]:
keywords = ['home', 'family', 'residen', 'mixed', 'mized', 'duplex', 'apartment', ' housing', 'condo', 'dwelling', 'tenant', 'affordable', 'units', 'townhouse']
pattern = '|'.join(keywords)
filtered1 = filter_cases_status[filter_cases_status['A_DESCRIPT'].str.contains(pattern, case=False, na=False)]

#use not pattern to avoid these words
keywords_avoid = ['addition','expand','storage', 'zoning', 'grading']
pattern_avoid = '|'.join(keywords_avoid)
filtered2 = filtered1[~filtered1['A_DESCRIPT'].str.contains(pattern_avoid, case=False, na=False)]

In [ ]:
#filter for all status dates after 2020
filtered2['A_STATUS_D'] = pd.to_datetime(filtered2['A_STATUS_D'])
filtered3 = filtered2[filtered2['A_STATUS_D'].dt.year>=2020]
# filtered3

In [ ]:
# function to extract unit quantity from description
def extract_units(description):
    description = re.sub(pattern=r'(\d+|\d{1,3}(,\d{3})*)(\s+[A-Za-z-]+){0,2}?\s*(SF|square feet|sq. ft.|sqft)', 
                         repl='', string=description, flags=re.IGNORECASE) # remove square footage descriptors

    match_pattern = r'(\d{1,3})(?:\s+[A-Za-z-]+){0,3}?\s+(units?|lots?|homes?|houses?|townhomes?|townhouses?|town homes?|town houses?|apartments?|duplex(?:es)?)'
    matches = re.findall(match_pattern, description, flags=re.IGNORECASE) # match residence types with their quantity

    return matches # returns a list of tuples in format (quantity, residence type)

In [ ]:
filtered3['match_results'] = filtered3['A_DESCRIPT'].apply(extract_units)
filtered3.head()